In [ ]:
%load_ext autoreload
%matplotlib inline
%autoreload 2
%config Completer.use_jedi = True
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc("figure", dpi=300)
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import ubergauss.tools as ut
df = ut.loadfile("/home/ubuntu/repos/cellsaw/application/results")

In [ ]:
import seaborn as sns 
from ubergauss.optimization import df_remove_duplicates as clean
df_clean = clean(df)
sns.pairplot(df_clean, hue = 'score')

In [ ]:

- one interesting thing to look at is dimensionality 
  of the projection... do a plot dim vs output.score.score()